In [ ]:
import sqlite3

def init_db():
    conn = sqlite3.connect('tasks.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY,
            description TEXT NOT NULL,
            status INTEGER NOT NULL CHECK (status IN (0, 1))
        )
    ''')
    conn.commit()
    conn.close()

if __name__ == '__main__':
    init_db()


In [ ]:
from flask import Flask, request, render_template, redirect, url_for
import sqlite3

app = Flask(__name__)

def get_db_connection():
    conn = sqlite3.connect('tasks.db')
    conn.row_factory = sqlite3.Row
    return conn

In [ ]:
@app.route('/')
def index():
    conn = get_db_connection()
    tasks = conn.execute('SELECT * FROM tasks').fetchall()
    conn.close()
    return render_template('home.html', tasks=tasks)

In [ ]:
@app.route('/create', methods=('GET', 'POST'))
def create():
    if request.method == 'POST':
        description = request.form['description']
        status = request.form['status']

        conn = get_db_connection()
        conn.execute('INSERT INTO tasks (description, status) VALUES (?, ?)',
                     (description, status))
        conn.commit()
        conn.close()
        return redirect(url_for('index'))

    return render_template('create.html')


In [ ]:
@app.route('/update/<int:id>', methods=('GET', 'POST'))
def update(id):
    conn = get_db_connection()
    task = conn.execute('SELECT * FROM tasks WHERE id = ?', (id,)).fetchone()

    if request.method == 'POST':
        description = request.form['description']
        status = request.form['status']

        conn.execute('UPDATE tasks SET description = ?, status = ? WHERE id = ?',
                     (description, status, id))
        conn.commit()
        conn.close()
        return redirect(url_for('index'))

    return render_template('update.html', task=task)

In [6]:

def delete(id):
    conn = get_db_connection()
    conn.execute('DELETE FROM tasks WHERE id = ?', (id,))
    conn.commit()
    conn.close()
    return redirect(url_for('index'))



In [16]:
!pip install flask


In [27]:
from jinja2 import Template


In [18]:
template_string = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Task List</title>
</head>
<body>
    <h1>Task List</h1>
    <a href="{{ url_for('create') }}">Create New Task</a>
    <ul>
        {% for task in tasks %}
        <li>
            {{ task.description }} - {% if task.status == 1 %}Complete{% else %}Incomplete{% endif %}
            <a href="{{ url_for('update', id=task.id) }}">Edit</a>
            <form action="{{ url_for('delete', id=task.id) }}" method="post" style="display:inline;">
                <button type="submit">Delete</button>
            </form>
        </li>
        {% endfor %}
    </ul>
</body>
</html>
"""

In [19]:
template_string = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Create Task</title>
</head>
<body>
    <h1>Create Task</h1>
    <form method="post">
        <label for="description">Description</label>
        <input type="text" id="description" name="description" required>
        <label for="status">Status</label>
        <select id="status" name="status">
            <option value="0">Incomplete</option>
            <option value="1">Complete</option>
        </select>
        <button type="submit">Create</button>
    </form>
</body>
</html>
"""

In [20]:
template_string = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Update Task</title>
</head>
<body>
    <h1>Update Task</h1>
    <form method="post">
        <label for="description">Description</label>
        <input type="text" id="description" name="description" value="{{ task.description }}" required>
        <label for="status">Status</label>
        <select id="status" name="status">
            <option value="0" {% if task.status == 0 %}selected{% endif %}>Incomplete</option>
            <option value="1" {% if task.status == 1 %}selected{% endif %}>Complete</option>
        </select>
        <button type="submit">Update</button>
    </form>
</body>
</html>
"""

In [22]:
template = Template(template_string)

In [25]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
